# ssd_people head detector inference demo

This is a brief tutorial that shows how to use our Single Shot Multibox Detector (SSD) based detector for head detection on images.

## 0. Setup
First, we import the libraries required to load images and setup the SSD head detector for inference. This model has been trained using [Pierluigi Ferrari's SSD implementation](https://github.com/pierluigiferrari/ssd_keras/), of which we include the required code in this repository. We refer you to the original implementation in case of an issue with that code.  

In [ ]:
from keras import backend as K
from keras.models import load_model
from keras.preprocessing import image
# from keras.optimizers import Adam
# from PIL import Image
from imageio import imread
import numpy as np
from matplotlib import pyplot as plt

# from models.keras_ssd512 import ssd_512
from keras_loss_function.keras_ssd_loss import SSDLoss
from keras_layers.keras_layer_AnchorBoxes import AnchorBoxes
from keras_layers.keras_layer_DecodeDetections import DecodeDetections
from keras_layers.keras_layer_DecodeDetectionsFast import DecodeDetectionsFast
from keras_layers.keras_layer_L2Normalization import L2Normalization

# from ssd_encoder_decoder.ssd_output_decoder import decode_detections, decode_detections_fast
# 
# from data_generator.object_detection_2d_data_generator import DataGenerator
# from data_generator.object_detection_2d_photometric_ops import ConvertTo3Channels
# from data_generator.object_detection_2d_geometric_ops import Resize
# from data_generator.object_detection_2d_misc_utils import apply_inverse_transforms

%matplotlib inline

This model uses a 512x512 input size, let's set our image size to that resolution size:

In [ ]:
# Set the image size.
img_height = 512
img_width = 512

## 1. Load a trained SSD model

### 1.1 Download the model's .h5 file
First, we download the model's file from the server to this repository's "data/models" folder. We provide a simple script which will do this task for you, which you can execute by running the next cell. NOTE: this script requires `curl`to be installed in your system. 

In case you prefer to download the model manually, you can download it using the following [link](https://drive.google.com/open?id=12cqKTPtQBAu780219hEbST7VwQuf6xDH).

In [ ]:
import subprocess
import os
os.chdir('./data/')
if 'download_model.sh' in os.listdir(os.getcwd()):
    print("Downloading model...")
    subprocess.call("./download_model.sh")
    print("Download finished.")

### 1.2 Load the model
Let's load the downloaded model. Make sure the 'weights_path' is correctly set to the model's file path.  

In [ ]:
# TODO: Set the path to the `.h5` file of the model
weights_path = './data/models/ssd512-hollywood-trainval-bs_16-lr_1e-05-scale_pascal-epoch-187.h5'

# Create an SSDLoss object in order to pass that to the model loader
ssd_loss = SSDLoss(neg_pos_ratio=3, n_neg_min=0, alpha=1.0)

# Clear previous models from memory.
K.clear_session() 

# Finally load the model
model = load_model(weights_path, custom_objects={'AnchorBoxes': AnchorBoxes,
                                                 'L2Normalization': L2Normalization,
                                                 'DecodeDetections': DecodeDetections,
                                                 'compute_loss': ssd_loss.compute_loss})

## 2. Load example images

Load some images for which you'd like the model to make predictions.

In [ ]:
# Original images array
orig_images = [] 
# Resized images array
input_images = []

# We'll only load one image in this example.
img_path = 'examples/people_drinking.jpg'
# img_path = 'examples/fish_bike.jpg'
# img_path = 'data/datasets/hollywood-heads/keyframes/Bruce_Almighty-frame-132378.png'

# Load the original image (used to display results)
orig_images.append(imread(img_path))
# orig_images.append(image.load_img(img_path))
# Load the image resized to the model's input size
img = image.load_img(img_path, target_size=(img_height, img_width))
img = image.img_to_array(img)
input_images.append(img)
input_images = np.array(input_images)

## 3. Make predictions

In [ ]:
y_pred = model.predict(input_images)
y_pred

`y_pred` contains a fixed number of predictions per batch item (200 if you use the original model configuration), many of which are low-confidence predictions or dummy entries. We therefore need to apply a confidence threshold to filter out the bad predictions. Set this confidence threshold value how you see fit.

In [ ]:
confidence_threshold = 0.3

# Perform confidence thresholding.
y_pred_thresh = [y_pred[k][y_pred[k,:,1] > confidence_threshold] for k in range(y_pred.shape[0])]

# Convert the predictions for the original image.
# y_pred_thresh_inv = apply_inverse_transforms(y_pred_thresh, batch_inverse_transforms)

np.set_printoptions(precision=2, suppress=True, linewidth=90)
print("Predicted boxes:\n")
print('   class   conf xmin   ymin   xmax   ymax')
print(y_pred_thresh[0])

## 4. Visualize the predictions

We just resized the input image above and made predictions on the distorted image. We'd like to visualize the predictions on the image in its original size though, so below we'll transform the coordinates of the predicted boxes accordingly.

In [ ]:
# Display the image and draw the predicted boxes onto it.

# Set the colors for the bounding boxes
colors = plt.cm.hsv(np.linspace(0, 1, 21)).tolist()
classes = ['background', 'head']

plt.figure(figsize=(20,12))
plt.imshow(orig_images[0])

current_axis = plt.gca()

for box in y_pred_thresh[0]:
    # Transform the predicted bounding boxes for the 512x512 image to the original image dimensions.
    xmin = box[2] * np.array(orig_images[0]).shape[1] / img_width
    ymin = box[3] * np.array(orig_images[0]).shape[0] / img_height
    xmax = box[4] * np.array(orig_images[0]).shape[1] / img_width
    ymax = box[5] * np.array(orig_images[0]).shape[0] / img_height
    color = colors[int(box[0])]
    label = '{}: {:.2f}'.format(classes[int(box[0])], box[1])
    current_axis.add_patch(plt.Rectangle((xmin, ymin), xmax-xmin, ymax-ymin, color=color, fill=False, linewidth=2))  
    current_axis.text(xmin, ymin, label, size='x-large', color='white', bbox={'facecolor':color, 'alpha':1.0})
